In [51]:
import numpy as np
from Orange.data import Table, Domain, ContinuousVariable, DiscreteVariable

def series2descriptor(d):
    if d.dtype is np.dtype("float") or d.dtype is np.dtype("int"):
        return ContinuousVariable(str(d.name))
    else:
        t = d.unique()
        t.sort()
        return DiscreteVariable(str(d.name), list(t.astype("str")))

def df2domain(df):
    featurelist = [series2descriptor(df.iloc[:,col]) for col in range(len(df.columns))]
    return Domain(featurelist)

def df2table(df):
    tdomain = df2domain(df)
    ttables = [series2table(df.iloc[:,i], tdomain[i]) for i in range(len(df.columns))]
    ttables = np.array(ttables).reshape((len(df.columns),-1)).transpose()
    return Table(tdomain , ttables)

def series2table(series, variable):
    print(series)
    if series.dtype is np.dtype("int") or series.dtype is np.dtype("float"):
        series = series.values[:, np.newaxis]
        return Table(series)
    else:
        series = series.astype('category').cat.codes.reshape((-1,1))
        return Table(series)
    
def column2df(col):
    if type(col.domain[0]) is Orange.feature.Continuous:
        return (col.domain[0].name, pd.Series(col.to_numpy()[0].flatten()))
    else:
        tmp = pd.Series(np.array(list(col)).flatten())  # type(tmp) -> np.array( dtype=list (Orange.data.Value) )
        tmp = tmp.apply(lambda x: str(x[0]))
        return (col.domain[0].name, tmp)

def table2df(tab):
    # Orange.data.Table().to_numpy() cannot handle strings
    # So we must build the array column by column,
    # When it comes to strings, python list is used
    series = [tab[i] for i in range(len(tab.domain))]
    series_name = [i[0] for i in series]  # To keep the order of variables unchanged
    series_data = dict(series)
    print(series_data)
    return pd.DataFrame(series_data, columns=series_name)

In [40]:
import Orange
tab = Orange.data.Table("lenses")
print(tab.domain)
series = [tab[i] for i in range(len(tab.domain))]
series

[age, prescription, astigmatic, tear_rate | lenses]


[[young, myope, no, reduced | none],
 [young, myope, no, normal | soft],
 [young, myope, yes, reduced | none],
 [young, myope, yes, normal | hard],
 [young, hypermetrope, no, reduced | none]]

In [41]:
import pandas as pd
series_name = [i[0] for i in series]  # To keep the order of variables unchanged
print(series_name)
#series_data = dict(series)
#print(series_data)
df = pd.DataFrame(series)
df

[Value('age', young), Value('age', young), Value('age', young), Value('age', young), Value('age', young)]


,0,1,2,3,4
0,2.0,1.0,0.0,1.0,1.0
1,2.0,1.0,0.0,0.0,2.0
2,2.0,1.0,1.0,1.0,1.0
3,2.0,1.0,1.0,0.0,0.0
4,2.0,0.0,0.0,1.0,1.0


In [9]:
df = table2df(data)

AttributeError: 'Table' object has no attribute 'select'

In [59]:
data = {'Name':['tom', 'e2', 'r2', 't2'],'Age':[28,34,29,42]}
df1 = pd.DataFrame(data,dtype=np.int)
print(df1)
df1.dtypes

  Name  Age
0  tom   28
1   e2   34
2   r2   29
3   t2   42


Name    object
Age      int32
dtype: object

In [60]:
tab = df2table(df1)

0    tom
1     e2
2     r2
3     t2
Name: Name, dtype: object


AttributeError: 'Series' object has no attribute 'reshape'

In [58]:
tab

[[2.000, 28.000],
 [2.000, 34.000],
 [2.000, 29.000],
 [2.000, 42.000]
]